## Recognition using OpenCV

These are the basic steps for getting plate numbers:

- Detect the plate on the car 
    - use contor in OpenCV to find rectangular objects
- Crop the image to contain only the license plate, using OpenCV
- Use Optical Character Recognition to detect the numbers on the plate 

In [47]:
import cv2
import imutils
import numpy as np
import pytesseract

### Detect the plate on the car

#### Resize the image and greyscale it

Creates consistency in the images, and removes details we don't need

In [57]:
#I don't want to add tesseract to my path, so telling it the command to the executable here

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [41]:
#we are reading in the img in color, may change this later to greyscale for simpler code
#then resizing it to have consistent size for images

pic = cv2.imread(r'C:\Users\Jrive\Desktop\car.jpeg', cv2.IMREAD_COLOR)
pic = cv2.resize(pic, (600, 400))

#converting it to greyscale
grey = cv2.cvtColor(pic, cv2.COLOR_BGR2GRAY)

#### Reduce noise and detect edges

A link to more on the parameters for bilateralFilter:
https://docs.opencv.org/master/d4/d86/group__imgproc__filter.html#ga9d7064d478c95d60003cf839430737ed

It is used to reduce noise, and is an improvement on Gaussian filters.
Increasing the last two values will increase blurring.

Canny is an algorithm used for edge detection. The two values that are passed in mean that only edge with an intensity gradient between those numberes will be displayed. 

In [49]:
grey = cv2.bilateralFilter(grey, 13, 15, 15)
edges = cv2.Canny(grey, 30, 200)

#### Lets take a look at what we have done so far

In [50]:
#taking a look at the reduced noise pic and the detected edges

# cv2.imshow("resized original", pic)
# cv2.imshow("reduced noise", grey)
# cv2.imshow("detected edges", edges)

# cv2.waitKey(0)
# cv2.destroyAllWindows()

#### Finding the contors in the image

This is going to look for the rectangle shape of a license plate

In [51]:
# we are passing in the image, the contour retrival mode, and contour approximation
#method
#RETR_TREE, our retrival mode, keeps all contours and constructs a heirarchy of contours
#CHAIN_APPROX_SIMPLE compresses horizontal, vertical, and diagonal segments and leaves 
#their endpoints, so a rectangle would be encoded with 4 points
contours = cv2.findContours(edges.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#this is parsing the contours
contours = imutils.grab_contours(contours)
#sorts the list of contours by the largest area of contours
contours = sorted(contours, key = cv2.contourArea, reverse=True)[:10]
screenCnt = None

In [52]:
#now we are going to sort through the ten largest contours
#and identify all rectangle contours with four sides and a closed figure

for c in contours:
    #finds the perimeter of the contour if it is a closed figure
    p = cv2.arcLength(c, True)
    #this approximates a polygon (our rectangle) with less vertices, keeping it within
    #a specified precision
    # that is our second parameter, again the last is to ensure it is a closed figure
    approx = cv2.approxPolyDP(c, 0.018 * p, True)
    #if our approx of the contour has four points, we stop and assume
    # that we have found our screen
    print(len(approx))
    if len(approx) == 4:
        screenCnt = approx
        break
        
#I need to add in code to handle errors from the above loop, will after getting skeleton
#will add comments for understanding soon 

if screenCnt is None:
    detected = 0
    print ("No contour detected")
else:
     detected = 1

if detected == 1:
    cv2.drawContours(pic, [screenCnt], -1, (0, 0, 255), 3)

8
9
7
7
4


#### Masking everything in the image that isn't our rectangle

In [53]:
#creating an empty array in the shape of our image, in the uint8 datatype specifically
mask = np.zeros(grey.shape, np.uint8)
#drawing our contour onto the mask, using the vector identified in the for loop above
#in 255 color, the -1 meaning contour interiors are also drawn
new_image = cv2.drawContours(mask, [screenCnt], 0, 255, -1)
#combines the picture with our mask
new_image = cv2.bitwise_and(pic, pic, mask=mask)

In [19]:
cv2.imshow("Masked image", new_image)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### Crop so that only license plate is in image

In [54]:
#get all points that are colored in the mask
(x, y) = np.where(mask == 255)
#get the minimum and maximum of these values so we can crop
(topx, topy) = (np.min(x), np.min(y))
(bottomx, bottomy) = (np.max(x), np.max(y))
#use those measurements to get the license plate from the rescaled, greyscale image
cropped = grey[topx:bottomx+1, topy:bottomy+1]

In [55]:
cv2.imshow("Cropped image", cropped)

cv2.waitKey(0)
cv2.destroyAllWindows()

#### Now we use pytesseract to read the plate

In [58]:
text = pytesseract.image_to_string(cropped, config = '--psm 11')
print(text)

CZ20FSE

